## Computational ressources

In [14]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 

cluster.scale(204) 

from dask.distributed import Client
client = Client(cluster)

client

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


Client Scheduler: tcp://172.30.100.4:43597 Dashboard: http://172.30.100.4:43005/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [15]:
!squeue -u albert7a

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           7483471      allc make_pro albert7a PD       0:00      1 (Priority)
           7483472      allc make_pro albert7a PD       0:00      1 (Priority)
           7483473      allc make_pro albert7a PD       0:00      1 (Priority)
           7483474      allc make_pro albert7a PD       0:00      1 (Priority)
           7483475      allc make_pro albert7a PD       0:00      1 (Resources)
           7483476      allc make_pro albert7a PD       0:00      1 (Resources)
           7483477      allc make_pro albert7a PD       0:00      1 (Resources)
           7483478      allc make_pro albert7a PD       0:00      1 (Resources)


In [16]:
import time
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)
print(str(nb_workers)+' are ready !')

70 are ready !


In [17]:
!squeue -u albert7a

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           7483474      allc make_pro albert7a PD       0:00      1 (Priority)
           7483475      allc make_pro albert7a PD       0:00      1 (Priority)
           7483476      allc make_pro albert7a PD       0:00      1 (Priority)
           7483477      allc make_pro albert7a PD       0:00      1 (Priority)
           7483478      allc make_pro albert7a PD       0:00      1 (Priority)
           7483471      allc make_pro albert7a  R       0:43      1 n2057
           7483472      allc make_pro albert7a  R       0:43      1 n2298
           7483473      allc make_pro albert7a  R       0:43      1 n2299


## Librairies and customization

In [18]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
from datetime import date
today=date.today()

import sys
sys.path.insert(0,'/home/albert7a/git/xscale')
import xscale

import xscale.spectral.fft as xfft
from xscale.spectral.tools import plot_spectrum
import xscale.signal.generator as xgen

import matplotlib.pyplot as plt
params = {'figure.figsize' : (15, 8),'legend.fontsize': 16,'xtick.labelsize':16,'ytick.labelsize':16,'axes.labelsize':16,'font.size':16}
plt.rcParams.update(params)


In [19]:
tfilenames_FMA = []

for month in ['02','03','04']:
    if month in ['12','01','02','03','04','05','06']:
        case='BLBT02X'
    else:
        case='BLBT02'

    if month in ['07','08','09','10','11','12']:
        year='2009'
    else:
        year='2010'
    
    data_dir = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-'+str(case)+'-S/'
    tfilenames_month = sorted(glob.glob(data_dir+'*/eNATL60-'+str(case)+'_1h_*_gridT-2D_'+str(year)+str(month)+'??-'+str(year)+str(month)+'??.nc'))
    tfilenames_FMA=tfilenames_FMA+tfilenames_month

In [20]:
%time dsFMA=xr.open_mfdataset(tfilenames_FMA,parallel=True)['sossheig']


CPU times: user 1min 6s, sys: 1min 20s, total: 2min 26s
Wall time: 4min 49s


In [21]:
dsFMA_reg=dsFMA[:,1950:2050,3950:4150].chunk({'x':1,'y':1,'time_counter':2136})

In [ ]:
NPADDING = 2
%time SPtime_FMA_reg = xfft.fft(dsFMA_reg, dim='time_counter', dx=1., detrend='mean',nfft=dsFMA_reg.shape[0]*NPADDING,tapering=True)


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/xarray/core/dataarray.py:218: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  elif isinstance(data, pd.Panel):


In [ ]:
SPtime_FMA_reg

In [ ]:
%time MEANPSD_FMA=xfft.psd(SPtime_FMA_reg).mean(dim='x').mean(dim='y').load()


In [ ]:
freqs_FMA=MEANPSD_FMA.f_time_counter.values
freqs_FMA = freqs_FMA[NPADDING::] 


In [ ]:
fig = plt.figure(facecolor='white')
ax = fig.add_subplot(111)

ax.loglog(freqs_FMA,MEANPSD_FMA[NPADDING::],'k',label='winter')
ax.loglog(freqs_ASO,MEANPSD_ASO[NPADDING::],'r',label='summer')
ax.set_xlim(1e-3,1)
ax.grid(which='both',axis='both')
plt.xlabel('Frequency (cpd)')
plt.ylabel('PSD (m2/cpd)')
plt.legend()
plt.title('Temporal Spectrum of SSH in the box')